In [3]:
import numpy as np
import random
from collections import defaultdict

In [2]:
# map + 
# agents
# boxes
# step

In [153]:
class env:
    def __init__(self, 
                 map_version, 
                 map_size, 
                 step_limit = 240, 
                 preparation = 0.4,
                 agents_number = [2, 2] , 
                 boxes_number = 4, 
                 debug = 0, 
                 use_seed = False, 
                 seed = 42):
        
        # self.seed = seed
        # self.use_seed = use_seed
        # if self.use_seed:
        #     random.seed(self.seed)
            

        self.debug = debug
        self.map_size = map_size
        self.base_step_limit = step_limit
        self.step_limit = self.base_step_limit
        self.box_amount = boxes_number
        self.seakers_amount = agents_number[0]
        self.hiders_amount = agents_number[1]
        
        # self.valid_rotates = ['-90 deg', '-45 deg', 'pass', '+45 deg', '+90 deg']
        self.valid_rotates = [-2, -1, 0, 1, 2]
        
        # self.valid_moves = ['forward', 'backward', 'pass']
        self.valid_moves = [0, 1, 2]
        
        # self.valid_interactions = ['pass', 'lock', 'unlock']
        self.valid_interactions = [0, 1, 2]
        
        self.move_map = {0:[-1,0], 
                         1:[1,0], 
                         2:[0,-1], 
                         3:[0,1], 
                         4:[0,0]}
        
        if map_version == 1:
            self.create_empty_map_v1()
        self.create_boxes()
        self.initialize_agents()

    def create_empty_map_v1_with_walls(self):
        if isinstance(self.map_size, int):
            if self.map_size >=6:
                # create empty map with aaditional place for external walls
                self.map = np.zeros((self.map_size+2, self.map_size+2), dtype=int)
                # add external walls
                self.map[0], self.map[-1] = 1, 1
                for i in range(self.map_size+2):
                    self.map[i][0], self.map[i][-1] = 1, 1

                # add room, with side of half of the map size
                self.room_size = self.map_size//2
                self.map[-(self.room_size+1)][-(self.room_size+1):] = 1
                for j in range(self.room_size):
                    self.map[-(j+1)][-(self.room_size+1)]=1

                # add doors in both walls of the room
                self.map[-(self.room_size+1)][-2:-1] = 0
                self.map[-(self.room_size-1)][-(self.room_size+1)] = 0
                #return(map_obj)
            elif self.map_size < 6 and self.map_size >= 0:
                print('Error: map size is too small, correct size is 6+')
        else:
            print('Error: incorrect map size type: expected int')
            
    def create_empty_map_v1(self):
        if isinstance(self.map_size, int):
            if self.map_size >=6:
                # create empty map with
                self.map = np.zeros((self.map_size, self.map_size), dtype=int)

                # add room, with side of half of the map size
                self.room_size = self.map_size//2
                self.map[-(self.room_size+1)][-(self.room_size+1):] = 1
                for j in range(self.room_size):
                    self.map[-(j+1)][-(self.room_size+1)]=1

                # add doors in both walls of the room
                rand_1 = random.randint(-(new_env.room_size), -1)
                rand_2 = random.randint(-(new_env.room_size), -1)
                self.map[-(self.room_size+1)][rand_1] = 0
                self.map[rand_2][-(self.room_size+1)] = 0
                #return(map_obj)
            elif self.map_size < 6 and self.map_size >= 0:
                print('Error: map size is too small, correct size is 6+')
        else:
            print('Error: incorrect map size type: expected int')
            
    def create_boxes(self):
        self.boxes = defaultdict()
        for i in range(self.box_amount):
            
            filt = (self.map == 0)
            
            for j in self.boxes.keys():
                coords = self.boxes[j]['coords']
                filt[coords[0], coords[1]] = 0
                
            filt = np.array(filt, dtype = 'int')
            rnd_p = random.choice(np.argwhere(filt))
            self.boxes[i] = defaultdict()
            self.boxes[i]['coords'] = list(rnd_p)
            self.boxes[i]['locked'] = False
            self.boxes[i]['locked_team'] = None
            
    def initialize_agents(self):
        self.agents = defaultdict()
        self.agents['seekers'] = defaultdict()
        self.agents['hiders'] = defaultdict()
        
        for s in range(self.seakers_amount):
            filt = (self.map == 0)
            for i in self.boxes.keys():
                coords = self.boxes[i]['coords']
                filt[coords[0], coords[1]] = 0
            filt = np.array(filt, dtype = 'int')
            
            for agent_id in self.agents['seekers'].keys():
                ag_coords = self.agents['seekers'][agent_id]['coords']
                filt[ag_coords[0], ag_coords[1]] = 0
            
            new_agent_coords = rnd_p = random.choice(np.argwhere(filt))
            new_agent = defaultdict()
            new_agent['coords'] = new_agent_coords
            new_agent['sight'] = random.randint(0, 8)
            self.agents['seekers'][s] = new_agent
            
        for h in range(self.seakers_amount):
            filt = (self.map == 0)
            for i in self.boxes.keys():
                coords = self.boxes[i]['coords']
                filt[coords[0], coords[1]] = 0
            filt = np.array(filt, dtype = 'int')
            
            for agent_id in self.agents['seekers'].keys():
                ag_coords = self.agents['seekers'][agent_id]['coords']
                filt[ag_coords[0], ag_coords[1]] = 0
                
            for agent_id in self.agents['hiders'].keys():
                ag_coords = self.agents['hiders'][agent_id]['coords']
                filt[ag_coords[0], ag_coords[1]] = 0
                
            new_agent_coords = rnd_p = random.choice(np.argwhere(filt))
            new_agent = defaultdict()
            new_agent['coords'] = new_agent_coords
            new_agent['sight'] = random.randint(0, 8)
            self.agents['hiders'][h] = new_agent
                
    def move(self, agent_numb, move_code):
        
        if self.step_limit == 0:
            if self.debug:
                print('step limit reached, error')
        else:
            self.step_limit -= 1
            if self.debug:
                print(self.step_limit, 'steps left')
                
        if move_code not in self.valid_moves:
            if self.debug:
                print('icorrect move')
        else:
            agent_old_coords = self.agents_coords[agent_numb]
            move = self.move_map[move_code]
            agent_new_coords = l3 = [l+r for l,r in zip(agent_old_coords, move)]
            correct_move = True
            for coord in agent_new_coords:
                if (coord < 0) or (coord >= self.size):
                    if self.debug:
                        print('icorrect move')
                    correct_move = False
            other_coords = self.agents_coords[1-agent_numb]
            if agent_new_coords == other_coords:
                if self.debug:
                    print('icorrect move')
                correct_move = False
            if correct_move:
                self.map[agent_old_coords[0],agent_old_coords[1]] = 0
                self.map[agent_new_coords[0],agent_new_coords[1]] = -1 - agent_numb
                self.agents_coords[agent_numb] = agent_new_coords
                for res in range(self.number_of_resources):
                    if self.resources[res][agent_new_coords[0],agent_new_coords[1]] != 0:
                        self.agents_res_collected[agent_numb].append(res)
                        self.resources[res][agent_new_coords[0],agent_new_coords[1]] = 0
                        if res == self.agents_goals[agent_numb]:
                            self.agents_self_goal_collected[agent_numb]+=1
                            if self.debug:
                                print('collected self goal resource:', res)
                        elif res == self.agents_goals[1 - agent_numb]:
                            self.agents_othr_goal_collected[agent_numb]+=1
                            if self.debug:
                                print('collected other goal resource:', res)
                        else:
                            self.agents_nthr_goal_collected[agent_numb]+=1
                            if self.debug:
                                print('collected neither goal resource:', res)
                            
    # def observation(self, agent_numb, with_map = 0):
        
    
    def reward(self):
        # we need to define a reward
        reward = 0
        
        return reward

    def reset(self):
        
        if self.use_seed:
            random.seed(self.seed)
            
        self.step_limit = self.base_step_limit
        self.initialize_map()

In [175]:
# map_version, map_size, step_limit
new_env = env(1, 30)
print(new_env.map)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0

In [145]:
random.randint(-10, -1)

-4

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]]


In [109]:
print(new_env.boxes[1])

defaultdict(None, {'coords': [13, 4], 'locked': False, 'locked_team': None})


In [110]:
new_env.agents

defaultdict(None,
            {'seekers': defaultdict(None,
                         {0: defaultdict(None,
                                      {'coords': array([ 5, 11], dtype=int64),
                                       'sight': 2}),
                          1: defaultdict(None,
                                      {'coords': array([8, 9], dtype=int64),
                                       'sight': 1})}),
             'hiders': defaultdict(None,
                         {0: defaultdict(None,
                                      {'coords': array([13,  5], dtype=int64),
                                       'sight': 3}),
                          1: defaultdict(None,
                                      {'coords': array([18,  5], dtype=int64),
                                       'sight': 1})})})

In [112]:
new_env.room_size

10

In [142]:
new_rand = random.randint(-(new_env.room_size), -1)
new_env.map[-(new_env.room_size + 1),new_rand]

1

In [141]:
new_env.map[-(new_env.room_size)]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [3]:
buffer_ag_1 = []
buffer_ag_2 = []

def sample_from_buffer(buffer, batch_size=1):
    batch = random.sample(buffer,batch_size)
    inputs = [i[0] for i in batch]
    actions = [i[1] for i in batch]
    revards = [i[2] for i in batch]
    
    inputs = torch.FloatTensor(inputs)
    actions = torch.FloatTensor(actions)
    revards = torch.FloatTensor(revards)
    
    return inputs, actions, revards

In [10]:
map_map = new_env.map
print(map_map)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]]


In [13]:
box_map = np.zeros((20, 20), dtype=int)
box_map[2,2] = 1
print(box_map)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [29]:
29//2

14